In [15]:
import pickle as pk
import bs4 as bs
import requests

In [11]:
reports = 'https://github.com/fadeeva/EDGAR_research/raw/refs/heads/master/data/reports.pickle'
requests.get(reports, "reports.pickle")

<Response [200]>

In [12]:
with open('reports.pickle', 'rb') as f:
    reports = pk.load(f)

len(reports.keys())

496

In [13]:
reports['AAPL']

['https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019325000008/aapl-20241228.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000081/aapl-20240629.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000069/aapl-20240330.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000006/aapl-20231230.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000077/aapl-20230701.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000064/aapl-20230401.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000006/aapl-20221231.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/aapl-20220625.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/aapl-20220326.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/aapl-20211225.htm',
 'https://www.sec.gov/Archives/e

In [44]:
headers={'User-Agent':'Dummy Company jane.dow@dummy.com','Accept-Encoding':'gzip, deflate','Host':'www.sec.gov'}

response = requests.get('https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
                        headers=headers, allow_redirects=True)

soup = bs.BeautifulSoup(response.text, features='lxml')

/tmp/ipython-input-44-2022039756.py:6: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = bs.BeautifulSoup(response.text, features='lxml')


In [45]:
len(soup.text)

80920

In [46]:
corpus = soup.text.replace('\n', ' ')

In [47]:
len(corpus)

80918

In [50]:
corpus[-100:]

'n ParekhKevan ParekhSenior Vice President,Chief Financial OfficerApple Inc. | Q2 2025 Form 10-Q | 23'